In [1]:
import json
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the NLLB model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def translate_to_target_language(target_language_code):
    # Recursively process each item in the JSON object
    def process_item(item):
        if isinstance(item, dict):
            new_translations = {}  # Temporary dictionary to store new translations
            for key, value in item.items():
                if key.endswith("FR"):
                    # Get the corresponding English key (without the -FR suffix)
                    english_key = key[:-3]
                    if english_key in item:
                        english_text = item[english_key]
                        # Translate the English text to the target language
                        inputs = tokenizer(english_text, return_tensors="pt")
                        translated_tokens = model.generate(
                            **inputs, forced_bos_token_id=tokenizer.lang_code_to_id[f"{target_language_code.split('_')[0]}_Latn"], max_length=30
                        )
                        target_translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
                        # Create the new key for the translated text
                        new_key = f"{english_key}_{target_language_code}"
                        new_translations[new_key] = target_translation  # Store new translation in temporary dictionary
                        print(f"Translated {english_key} to {new_key}: {target_translation}")
                    else:
                        # # Issue a warning if there are any other sibling tags without an underscore
                        # for sibling_key in item.keys():
                        #     if '_' not in sibling_key and sibling_key != key:
                        #         print(f'Warning: Skipping translation for {key} due to sibling tag {sibling_key} without an underscore.')
                        #         break
                        continue

                # Recursively process nested items
                process_item(value)

            item.update(new_translations)  # Update original dictionary with new translations

        elif isinstance(item, list):
            for value in item:
                process_item(value)

    
    # Load the JSON file
    with open("HOTTP.json", "r", encoding="utf-8") as file:
        data = json.load(file)

    # Process the loaded data
    process_item(data)

    # Save the updated JSON data to a new file
    with open(f"HOTTP_translated_{target_language_code}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Translate to a specific target language (e.g., 'spa_Latn')
translate_to_target_language('spa_Latn')


Translated Remark to Remark_spa_Latn: Hay tres formas posibles de traducir el MT: (1) Expresión con una cláusula relativa implícita limitada al versículo 1, con el
Translated Remark to Remark_spa_Latn: Ver caso similar en Génesis 1.6
Translated Remark to Remark_spa_Latn: Los traductores que usan notas podrían añadir en una nota: la versión variante que dice "todos los animales de la tierra",
Translated Remark to Remark_spa_Latn: El significado del versículo 2 sería: "Cuando Dios terminó por el séptimo día toda su obra que había hecho, descans


KeyboardInterrupt: 